In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, Flatten, Dense, Dropout, 
                                     Embedding, LSTM, Bidirectional, Input, concatenate, 
                                     LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D)
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# -----------------------------
# Simulated data for demo
# -----------------------------
NUM_CLASSES = 5
MAX_LEN = 1000
VOCAB_SIZE = 5

np.random.seed(42)
X = np.random.randint(0, VOCAB_SIZE, (800, MAX_LEN))
y_multi = tf.keras.utils.to_categorical(np.random.randint(0, NUM_CLASSES, 800), NUM_CLASSES)
y_bin1 = np.random.randint(0, 2, 800)
y_bin2 = np.random.randint(0, 2, 800)
y_bin3 = np.random.randint(0, 2, 800)

X_train, X_val, y_train_multi, y_val_multi = train_test_split(X, y_multi, test_size=0.2)
_, _, y_train_bin1, y_val_bin1 = train_test_split(X, y_bin1, test_size=0.2)
_, _, y_train_bin2, y_val_bin2 = train_test_split(X, y_bin2, test_size=0.2)
_, _, y_train_bin3, y_val_bin3 = train_test_split(X, y_bin3, test_size=0.2)

# -----------------------------
# Model 1: 1D CNN (Multiclass)
# -----------------------------
cnn_model = Sequential([
    Embedding(VOCAB_SIZE, 64, input_length=MAX_LEN),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(NUM_CLASSES, activation='softmax')
])
cnn_model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train_multi, epochs=5, validation_data=(X_val, y_val_multi), verbose=1)

# -----------------------------
# Model 2: BiLSTM (Binary)
# -----------------------------
input_seq = Input(shape=(MAX_LEN,))
x = Embedding(VOCAB_SIZE, 64)(input_seq)
x = Bidirectional(LSTM(64, return_sequences=False))(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)
bilstm_model = Model(input_seq, output)
bilstm_model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model.fit(X_train, y_train_bin1, epochs=5, validation_data=(X_val, y_val_bin1), verbose=1)

# -----------------------------
# Model 3: CNN + BiLSTM (Binary)
# -----------------------------
input_seq = Input(shape=(MAX_LEN,))
x = Embedding(VOCAB_SIZE, 64)(input_seq)
cnn_out = Conv1D(64, 5, activation='relu')(x)
cnn_out = MaxPooling1D(2)(cnn_out)
lstm_out = Bidirectional(LSTM(64))(cnn_out)
dense = Dense(64, activation='relu')(lstm_out)
output = Dense(1, activation='sigmoid')(dense)
cnn_lstm_model = Model(input_seq, output)
cnn_lstm_model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
cnn_lstm_model.fit(X_train, y_train_bin2, epochs=5, validation_data=(X_val, y_val_bin2), verbose=1)

# -----------------------------
# Model 4: Transformer Encoder (Binary)
# -----------------------------
input_seq = Input(shape=(MAX_LEN,))
x = Embedding(VOCAB_SIZE, 64)(input_seq)
attn = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = LayerNormalization()(x + attn)
x = GlobalAveragePooling1D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)
transformer_model = Model(input_seq, output)
transformer_model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
transformer_model.fit(X_train, y_train_bin3, epochs=5, validation_data=(X_val, y_val_bin3), verbose=1)

# -----------------------------
# Evaluation
# -----------------------------
models = {
    "CNN": cnn_model,
    "BiLSTM": bilstm_model,
    "CNN+BiLSTM": cnn_lstm_model,
    "Transformer": transformer_model
}

print("\n=== Model Performance Summary ===")
for name, model in models.items():
    if name == "CNN":
        val_pred = np.argmax(model.predict(X_val), axis=1)
        true = np.argmax(y_val_multi, axis=1)
    else:
        val_pred = (model.predict(X_val) > 0.5).astype(int).flatten()
        true = [y_val_bin1, y_val_bin2, y_val_bin3][["BiLSTM","CNN+BiLSTM","Transformer"].index(name)]
    acc = accuracy_score(true, val_pred)
    print(f"{name:<15} Accuracy: {acc*100:.2f}%")

# -----------------------------
# Ensemble Voting (Weighted)
# -----------------------------
pred_cnn = np.argmax(cnn_model.predict(X_val), axis=1)
pred_bilstm = (bilstm_model.predict(X_val) > 0.5).astype(int).flatten()
pred_cnn_lstm = (cnn_lstm_model.predict(X_val) > 0.5).astype(int).flatten()
pred_trans = (transformer_model.predict(X_val) > 0.5).astype(int).flatten()

# Combine binary votes (majority)
ensemble_votes = np.round((pred_bilstm + pred_cnn_lstm + pred_trans) / 3)
ensemble_acc = accuracy_score(y_val_bin3, ensemble_votes)
print(f"\n✅ Ensemble Binary Accuracy (final stage): {ensemble_acc*100:.2f}%")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-10-28 16:49:17.827880: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 189ms/step - accuracy: 0.2110 - loss: 1.6092 - val_accuracy: 0.2188 - val_loss: 1.6089
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.1722 - loss: 1.6105 - val_accuracy: 0.2313 - val_loss: 1.6085
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 156ms/step - accuracy: 0.2186 - loss: 1.6090 - val_accuracy: 0.2313 - val_loss: 1.6080
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 157ms/step - accuracy: 0.2090 - loss: 1.6096 - val_accuracy: 0.2188 - val_loss: 1.6078
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 0.1948 - loss: 1.6097 - val_accuracy: 0.2188 - val_loss: 1.6078
Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 613ms/step - accuracy: 0.5306 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 561ms/step - accuracy: 0.4484 - loss: 0.6939 - val_accuracy: 0.4938 - val_loss: 0.6931
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 591ms/step - accuracy: 0.5096 - loss: 0.6930 - val_accuracy: 0.5125 - val_loss

In [18]:
import os
import tensorflow as tf

def save_trained_models(models_dict, save_dir="trained_models"):
    """
    Saves all trained models to disk.
    Args:
        models_dict: dict of {'model_name': model_object}
        save_dir: target directory to store .h5 files
    """
    os.makedirs(save_dir, exist_ok=True)
    for name, model in models_dict.items():
        path = os.path.join(save_dir, f"{name}.h5")
        model.save(path)
        print(f"✅ Saved {name} → {path}")
    print("\nAll models saved successfully!")

# Example usage (after training all 4 models)
models_dict = {
    "model1_cnn_multiclass": cnn_model,
    "model2_bilstm_binary": bilstm_model,
    "model3_cnn_bilstm_binary": cnn_lstm_model,
    "model4_transformer_binary": transformer_model
}

save_trained_models(models_dict)


✅ Saved model1_cnn_multiclass → trained_models/model1_cnn_multiclass.h5
✅ Saved model2_bilstm_binary → trained_models/model2_bilstm_binary.h5
✅ Saved model3_cnn_bilstm_binary → trained_models/model3_cnn_bilstm_binary.h5
✅ Saved model4_transformer_binary → trained_models/model4_transformer_binary.h5

All models saved successfully!


In [25]:

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout

# ======================================
# 🧩 STEP 2: Custom Transformer Block
# (needed to load transformer model)
# ======================================
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# ======================================
# 🧬 STEP 3: Load Saved Models
# ======================================
def load_saved_models(save_dir="trained_models"):
    model_files = {
        "cnn_multiclass": "model1_cnn_multiclass.h5",
        "bilstm_binary": "model2_bilstm_binary.h5",
        "cnn_bilstm_binary": "model3_cnn_bilstm_binary.h5",
        "transformer_binary": "model4_transformer_binary.h5"
    }

    models = {}
    for name, file in model_files.items():
        path = os.path.join(save_dir, file)
        if not os.path.exists(path):
            print(f"❌ Model not found: {path}")
            continue

        try:
            if "transformer" in name:
                models[name] = tf.keras.models.load_model(
                    path,
                    custom_objects={
                        "TransformerBlock": TransformerBlock,
                        "MultiHeadAttention": tf.keras.layers.MultiHeadAttention,
                        "LayerNormalization": tf.keras.layers.LayerNormalization,
                        "Dropout": tf.keras.layers.Dropout,
                        "Dense": tf.keras.layers.Dense
                    },
                    compile=False
                )
            else:
                models[name] = tf.keras.models.load_model(path)

            print(f"✅ Loaded {name.upper()} → {path}")

        except Exception as e:
            print(f"⚠️ Error loading {name.upper()}: {e}")

    return models

# ======================================
# 🔡 STEP 4: DNA Sequence Preprocessing
# ======================================
def preprocess_dna(seq, maxlen=1000):
    seq = seq.upper().strip()
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    encoded = [mapping.get(ch, 0) for ch in seq]
    if len(encoded) < maxlen:
        encoded += [0] * (maxlen - len(encoded))
    else:
        encoded = encoded[:maxlen]
    return np.array(encoded).reshape(1, maxlen)

# ======================================
# 🧬 STEP 5: Predict Gene Type
# ======================================
def predict_gene_type(sequence, models):
    x = preprocess_dna(sequence)
    results = {}

    # Label names for CNN multiclass output
    gene_labels = ["PSEUDO", "BIOLOGICAL_REGION", "ncRNA", "OTHER"]

    # --- CNN Multiclass ---
    if "cnn_multiclass" in models:
        pred_cnn = models["cnn_multiclass"].predict(x, verbose=0)
        pred_class = int(np.argmax(pred_cnn))
        results["CNN_Multiclass"] = {
            "Predicted_Gene_Type": gene_labels[pred_class],
            "Confidence": float(np.max(pred_cnn))
        }

    # --- Binary Models ---
    binary_mapping = {
        "bilstm_binary": "PSEUDO",
        "cnn_bilstm_binary": "BIOLOGICAL_REGION",
        "transformer_binary": "ncRNA"
    }

    for key, label in binary_mapping.items():
        if key in models:
            pred = models[key].predict(x, verbose=0).flatten()[0]
            results[key.upper()] = {
                "Predicted_Gene_Type": label if pred > 0.5 else "OTHER",
                "Confidence": round(float(pred), 3)
            }

    return results

# ======================================
# 🚀 STEP 6: Run Prediction Example
# ======================================
models = load_saved_models()

dna_seq = input("\n🧬 Enter DNA Sequence: ").strip()
predictions = predict_gene_type(dna_seq, models)

print("\n========== 🧬 PREDICTION RESULTS ==========")
for model_name, info in predictions.items():
    print(f"{model_name}: {info}")



✅ Loaded CNN_MULTICLASS → trained_models/model1_cnn_multiclass.h5
✅ Loaded BILSTM_BINARY → trained_models/model2_bilstm_binary.h5
✅ Loaded CNN_BILSTM_BINARY → trained_models/model3_cnn_bilstm_binary.h5
⚠️ Error loading TRANSFORMER_BINARY: too many positional arguments



🧬 Enter DNA Sequence:  TTGTGATTATGTTATTTTACATGGCAAAAGGGACTTTGCAGCGGTGACTAACATAAGGACCTTGAGATGAGGAGATTATCCCCAGGAACATAATCACATGAGTTCTTAAACACAGGGGACATTTGCTAATGATGGCCCAAGGGAACTGACTATAGAAAAATGGAAAGAGAGTTGCTTATCAGCGTAAGAAACTTTTGGCCTGAGACGATGGGGTTTTCTAAATATAGGATCATGTAATCCACAAACAGAGATAGTTTGACTTCCTCTCTTCCTATTTGAATACACTTTATTTCTT



========== 🧬 PREDICTION RESULTS ==========
CNN_Multiclass: {'Predicted_Gene_Type': 'PSEUDO', 'Confidence': 0.20418643951416016}
BILSTM_BINARY: {'Predicted_Gene_Type': 'PSEUDO', 'Confidence': 0.5}
CNN_BILSTM_BINARY: {'Predicted_Gene_Type': 'OTHER', 'Confidence': 0.498}
